In [5]:
import time
import requests
# from kafka import KafkaConsumer, TopicPartition

# Flink REST API 地址
flink_api_url = 'http://localhost:8082'

# Kafka 相关配置
kafka_bootstrap_servers = 'localhost:9092'
kafka_topic = 'my-topic'
kafka_group_id = 'my-group'

# 获取任务 source 的数据处理速率
def get_task_source_rate():
    # 发送 GET 请求获取任务列表
    response = requests.get(flink_api_url + '/jobs')
    response_json = response.json()

    # 遍历任务列表，获取每个任务的 source 数据处理速率
    for job in response_json['jobs']:
        job_id = job['id']
        response = requests.get(flink_api_url + f'/jobs/{job_id}/vertices')
        response_json = response.json()
        for vertex in response_json['vertices']:
            if vertex['type'] == 'Source':
                vertex_id = vertex['id']
                response = requests.get(flink_api_url + f'/jobs/{job_id}/vertices/{vertex_id}/metrics')
                response_json = response.json()
                for metric in response_json['metrics']:
                    if metric['id'] == 'numBytesInLocal':
                        rate = metric['value'] / 30  # 计算数据处理速率，单位为 B/s
                        print(f'Job {job_id}, Source {vertex_id}: {rate} B/s')

# 获取 Kafka Lag 信息并计算数据到达率
# def get_kafka_lag_rate():
#     consumer = KafkaConsumer(bootstrap_servers=kafka_bootstrap_servers, group_id=kafka_group_id)
#     partitions = consumer.partitions_for_topic(kafka_topic)
#     if partitions is None:
#         return
#     for partition in partitions:
#         tp = TopicPartition(kafka_topic, partition)
#         consumer.assign([tp])
#         consumer.seek_to_end(tp)
#         end_offset = consumer.position(tp)
#         consumer.seek_to_beginning(tp)
#         start_offset = consumer.position(tp)
#         lag = end_offset - start_offset
#         rate = lag / 30  # 计算数据到达率，单位为条/秒
#         print(f'Partition {partition}: {rate} records/s')

# 主程序
if __name__ == '__main__':
    while True:
        # 获取任务 source 的数据处理速率
        get_task_source_rate()

        # 获取 Kafka Lag 信息并计算数据到达率
#         get_kafka_lag_rate()

        # 等待 30s
        time.sleep(30)


KeyError: 'vertices'

In [3]:
while True:
    # 获取任务 source 的数据处理速率
    get_task_source_rate()

    # 获取 Kafka Lag 信息并计算数据到达率
#         get_kafka_lag_rate()

    # 等待 30s
    time.sleep(30)

In [9]:
response = requests.get(flink_api_url + '/jobs')
response_json = response.json()

print(response_json)

{'jobs': [{'id': '53d81da5253844501806c95af939db6b', 'status': 'CANCELED'}, {'id': 'b88d39552dda87f0479f05c31a5e2a31', 'status': 'FAILED'}, {'id': 'cd0b818c3c5833e8f44e08117cc973c3', 'status': 'RUNNING'}, {'id': '8a712689e0515bc72c4269c0269895af', 'status': 'CANCELED'}, {'id': '0f4ab343e711b5ea4da42f648f4cab27', 'status': 'CANCELED'}]}


In [1]:
print(1)

1


In [10]:
for job in response_json['jobs']:
    if job['status'] != 'RUNNING':
        continue
    job_id = job['id']
    job = requests.get(flink_api_url + f'/jobs/{job_id}/vertices')
    job_json = job.json()
    for vertex in job_json['vertices']:
        if vertex['type'] == 'Source':
            vertex_id = vertex['id']
            response = requests.get(flink_api_url + f'/jobs/{job_id}/vertices/{vertex_id}/metrics')
            response_json = response.json()
            for metric in response_json['metrics']:
                if metric['id'] == 'numBytesInLocal':
                    rate = metric['value'] / 30  # 计算数据处理速率，单位为 B/s
                    print(f'Job {job_id}, Source {vertex_id}: {rate} B/s')

KeyError: 'vertices'

In [11]:
job_json

{'errors': ['Unable to load requested file /jobs/cd0b818c3c5833e8f44e08117cc973c3/vertices.']}

In [27]:
import flink_rest_client
from flink_rest_client.v1.client import FlinkRestClientV1

In [28]:
remote_client: FlinkRestClientV1 = flink_rest_client.FlinkRestClient.get(host="localhost", port=8082)
local_client: FlinkRestClientV1 = flink_rest_client.FlinkRestClient.get(host="localhost", port=8081)

In [30]:
remote_client.jobs.all()

[{'id': '71304e3b1a663431746c7f93748a0bd0', 'status': 'CANCELED'},
 {'id': 'fa03a3b8761ba20dda1986c7f37d9830', 'status': 'RUNNING'},
 {'id': '6a960016374a108948ac41a189eddc09', 'status': 'CANCELED'},
 {'id': '7dfedd20ca09435c6e9d5ae8fe885d54', 'status': 'CANCELED'}]

In [40]:
for job in local_client.jobs.all():
    if job['status']=='RUNNING':
        query = 'sum by(task_name) (flink_taskmanager_job_task_operator_numRecordsInPerSecond{{job_id="{}"}})'.format(job['id'])
        result = prom.custom_query(query)
        print(result)

[{'metric': {'task_name': 'Sink:_kafka_hotitems'}, 'value': [1686153394.498, '5006.5']}, {'metric': {'task_name': 'Source:_HotPagesGenerator'}, 'value': [1686153394.498, '0']}]


In [39]:
for job in remote_client.jobs.all():
    if job['status']=='RUNNING':
        query = 'sum by(task_name) (flink_taskmanager_job_task_operator_numRecordsInPerSecond{{job_id="{}"}})'.format(job['id'])
        result = prom.custom_query(query)
        print(result)

[{'metric': {'task_name': 'Map'}, 'value': [1686153375.537, '5004.85']}, {'metric': {'task_name': 'SlidingEventTimeWindows'}, 'value': [1686153375.537, '1244.5499999999997']}, {'metric': {'task_name': 'Timestamps_Watermarks'}, 'value': [1686153375.537, '1245.25']}, {'metric': {'task_name': 'Projection'}, 'value': [1686153375.537, '1244.15']}, {'metric': {'task_name': 'Sink:_Print_to_Std__Out'}, 'value': [1686153375.537, '2000.1999999999998']}, {'metric': {'task_name': 'Source:_kafka_hotitems'}, 'value': [1686153375.537, '0']}, {'metric': {'task_name': 'KeyedProcess'}, 'value': [1686153375.537, '2000']}, {'metric': {'task_name': 'Filter'}, 'value': [1686153375.537, '6249.083333333333']}]


In [14]:
from prometheus_api_client import PrometheusConnect

prom = PrometheusConnect(url="http://localhost:9090")

In [45]:
query = 'sum by(task_name) (flink_taskmanager_job_task_operator_numRecordsInPerSecond{job_id="7127565bf8c16d1006ef729495ef04d4"})'
result = prom.custom_query(query)

In [48]:
result[0]['value'][1]

'1308.3166666666666'

In [26]:
query = 'sum by(task_name) (flink_taskmanager_job_task_operator_numRecordsInPerSecond{})'
prom.custom_query(query)

[{'metric': {'task_name': 'Sink:_kafka_hotitems'},
  'value': [1686152454.634, '5000']},
 {'metric': {'task_name': 'Source:_HotPagesGenerator'},
  'value': [1686152454.634, '0']},
 {'metric': {'task_name': 'Map'},
  'value': [1686152454.634, '4994.833333333334']},
 {'metric': {'task_name': 'SlidingEventTimeWindows'},
  'value': [1686152454.634, '1251.4333333333334']},
 {'metric': {'task_name': 'Timestamps_Watermarks'},
  'value': [1686152454.634, '1253.9166666666667']},
 {'metric': {'task_name': 'Projection'}, 'value': [1686152454.634, '1252']},
 {'metric': {'task_name': 'Sink:_Print_to_Std__Out'},
  'value': [1686152454.634, '2000.2']},
 {'metric': {'task_name': 'Source:_kafka_hotitems'},
  'value': [1686152454.634, '0']},
 {'metric': {'task_name': 'KeyedProcess'}, 'value': [1686152454.634, '2000']},
 {'metric': {'task_name': 'Filter'},
  'value': [1686152454.634, '6246.0666666666675']}]

In [58]:
query = 'sum by(job_name) (flink_taskmanager_job_task_operator_records_lag_max{job_id="7b240762e89bc813a18b44b28d11fa13"})'
prom.custom_query(query)[0]['value'][1]

'78'